<a href="https://colab.research.google.com/github/geomaticajavl/GeoAI_Plugin/blob/main/ocr_ci__updated_agosto2025ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
!pip install PyPDF2
!apt-get install tesseract-ocr
!pip install pytesseract
!pip install tabula
!pip install openpyxl
!pip install odfpy
!pip install qrcode
!pip install easyocr
!apt install libtesseract-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libtesseract-dev is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [52]:
import openpyxl
import PyPDF2
import pytesseract
# Updated path for tesseract executable in Google Colab
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'
import easyocr
import gspread
import tabula
import os
import pandas as pd
from PIL import Image
import io
from io import BytesIO
from google.colab import files
from google.colab import drive
#drive.mount('/content/drive')
from odf.opendocument import OpenDocumentSpreadsheet
from odf.table import Table, TableRow, TableCell
from odf.text import P
import re
import cv2
import qrcode

In [53]:
#@title Chunk para carga de archivos
import google.colab.files

def subir_archivos():
    print("Cargar los archivos .jpg, .tiff, .png, .jpeg,.xlsx; .xls; .csv; ods")
    archivos_subidos = google.colab.files.upload()
        # Imprime un mensaje de confirmación
    for nombre_archivo in archivos_subidos.keys():
        print('Se ha subido el archivo "{nombre}" con longitud {longitud} bytes'.format(
            nombre=nombre_archivo, longitud=len(archivos_subidos[nombre_archivo])))
subir_archivos()

Cargar los archivos .jpg, .tiff, .png, .jpeg,.xlsx; .xls; .csv; ods


Saving Imagen de WhatsApp 2025-08-06.jpg.jpg to Imagen de WhatsApp 2025-08-06.jpg.jpg
Se ha subido el archivo "Imagen de WhatsApp 2025-08-06.jpg.jpg" con longitud 74273 bytes


In [50]:
#@title Función para limpiar la carpeta temporal /content/ de google colab

# Obtiene la ruta a la carpeta `content`
ruta_content = os.getcwd()

# Obtiene una lista de todos los archivos .xlsx de la carpeta /content/
archivos_jpg_png_csv_odf = [archivo for archivo in os.listdir(ruta_content)\
                 if archivo.endswith(".jpg") or archivo.endswith(".csv")\
                 or archivo.endswith(".xlsx") or archivo.endswith(".odf")]

# Elimina todos los archivos .xlsx de la carpeta `content`
for archivo in archivos_jpg_png_csv_odf:
    os.remove(os.path.join(ruta_content, archivo))

# Imprime un mensaje de confirmación
print("Los archivos .jpg/.tif/./odf.xlsx/.xls/.csv/.ods/ temporales han sido borrados")

Los archivos .jpg/.tif/./odf.xlsx/.xls/.csv/.ods/ temporales han sido borrados


In [56]:
#@title Modelo extractor en POO
# Construyo un objeto de lector (especifica el idioma)
reader = easyocr.Reader(['es'])

# Cargar la imagen using PIL and convert to bytes
try:
    image_path = '/content/Imagen de WhatsApp 2025-08-06.jpg.jpg' # Corrected filename
    # Use PIL to open and read the image as bytes
    with open(image_path, 'rb') as f:
        imagen_bytes = f.read()

    # Realizar OCR
    resultados = reader.readtext(imagen_bytes)

    # Imprimir los resultados
    for (bbox, texto, probabilidad) in resultados:
        print(f'Texto: {texto}, Probabilidad: {probabilidad}')

except FileNotFoundError:
    print(f"Error: The file {image_path} was not found.")
except Exception as e:
    print(f"An error occurred during OCR: {e}")

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Texto: Aalga, Probabilidad: 0.020752805074801924
Texto: Bolivarvana, Probabilidad: 0.08961190739864558
Texto: OE, Probabilidad: 0.033591348927465806
Texto: ee0, Probabilidad: 0.015785338055803195
Texto: Ced U L A, Probabilidad: 0.27397719417493377
Texto: 0e, Probabilidad: 0.7597421139024026
Texto: LDenidad, Probabilidad: 0.0708803518626738
Texto: V, Probabilidad: 0.9543763135185692
Texto: 14.964.511, Probabilidad: 0.9571108938180772
Texto: MM7ss, Probabilidad: 0.3635248910764222
Texto: Anaoelimeng, Probabilidad: 0.11158635454013383
Texto: Apeludos, Probabilidad: 0.5335008536253092
Texto: VALERA LEAL, Probabilidad: 0.8201181483789216
Texto: Oirecror, Probabilidad: 0.4599904620855379
Texto: Nohbres, Probabilidad: 0.36990116587119837
Texto: JAVIER ALEXANDER, Probabilidad: 0.9996854662322177
Texto: 24-06-81 SOLTERO, Probabilidad: 0.7596074121750493
Texto: F.Nacimiento, Probabilidad: 0.37089279091973776
Texto: EDo CIVIL, Probabilidad: 0.2408280285524795
Texto: 30-09-15 09-2025, Probabilidad

# Acción
Refactorizo aquí el código para extraer consistentemente los siguientes campos de los resultados del OCR y guardarlos en un archivo CSV y XLSX con los encabezados especificados: "F. Vencimiento", "F. Nacimiento", "Estado Civil", "Apellidos", "Nombres", "Número de Identificación". Asegúrese de que el formato de salida sea estable y coincida con el ejemplo proporcionado (F. Vencimiento; F. Nacimiento; Estado civil; Apellidos; Nombres): 30-09-15,24-06-81,SOLTERO,VALERA LEAL,ALEXANDER,14.964.511.

## Refino y refactorización de la lógica de extracción de datos en imagen de acuerdo a una ubicación lógica según las coordenadas de los datos que necesito

### Subtarea:
 Analizar el código de extracción actual (específicamente en la celda a0acfc48) para identificar áreas que puedan mejorarse y estabilizarse. Esto podría implicar refinar la agrupación de líneas, los patrones de coincidencia de texto y la lógica para asociar el texto extraído con los campos correctos.


**Prompt para futuras modificaciones/actualizaciones**:
Analyze the existing code in cell `a0acfc48` and identify areas for improvement to make the extraction more robust and stable.



In [57]:
# Analyze the current code's approach to extraction:

# 1. Line Grouping and line_tolerance:
# The approach of grouping text into lines based on a dynamic tolerance relative to average text height is a good starting point.
# However, average height might not be representative for images with varying font sizes or sections of text.
# A fixed tolerance or a tolerance based on the median height might be more stable.
# Also, the current implementation of line grouping might not correctly handle skewed or curved text lines.

# 2. Text Matching Patterns and Regular Expressions:
# - F. Vencimiento and F. Nacimiento: The regex r'(\d{2}[-/]\d{2}[-/]\d{2,4})' is reasonable for common date formats.
#   The logic for looking in the line above F.VENCIMIENTO is good.
# - Estado Civil: The regex r'(soltero|soltera|casado|casada|viudo|viuda)' is too restrictive. It only captures specific words and might miss variations or abbreviations.
#   The logic for checking the line above is also implemented.
# - Apellidos and Nombres: The approach of looking for text after keywords like "APELLIDOS" or "NOMBRES" on the same line is sensitive to variations in spacing and the presence of other text after the keyword.
#   The exclusion of "Anapelmeng" is specific to the example image and not a general solution.
#   The regex for cleaning (`[^a-zA-Z\s]+$`) might be too aggressive and remove valid parts of names.
# - Número de Identificación: The regex r"V\s*([\d.]+)" is good for capturing the number after 'V'.
#   The fallback to check the next line for a number is a good addition, but might incorrectly pick up other numerical data.

# 3. Association Logic:
# The logic primarily relies on keyword presence within lines and relative vertical position (current line or line above/below).
# This can be problematic when keywords are missed by OCR, or when the layout is significantly different.
# Horizontal positioning relative to keywords is attempted for names/surnames but could be refined.

# Identified areas for improvement:
# - More flexible Estado Civil extraction (consider a wider range of keywords or a different approach).
# - More robust Apellidos and Nombres extraction that is less sensitive to noise and variations in text after the keyword.
# - Refine the line grouping logic to handle potential skew and variations in text size more gracefully.
# - Enhance the association logic, potentially using relative bounding box positions more extensively, especially for fields like names and surnames that appear next to their labels.
# - Consider implementing a scoring mechanism or confidence threshold for extracted fields, especially when multiple potential matches are found.

print("Analysis of cell a0acfc48 completed. Potential areas for improvement identified.")

Analysis of cell a0acfc48 completed. Potential areas for improvement identified.


**Procedimiento -promptong engeenering-**:
Con base en el análisis, refine la lógica de extracción en la celda `a0acfc48` para abordar las debilidades identificadas, particularmente en la extracción de 'Estado Civil', 'Apellidos' y 'Nombres', y para mejorar la solidez de la agrupación y asociación de líneas.



In [58]:
# Refined extraction logic based on analysis

# Initialize a list to store the extracted data
extracted_data = []

# Assuming 'resultados' contains the output from easyocr.readtext()
# Iterate through the results to find the required information
f_vencimiento_value = None
f_nacimiento_value = None
edo_civil_value = None
apellidos_text = None
nombres_text = None
numero_identificacion_text = None

# Store results and their approximate vertical positions
results_with_y = [(item, (item[0][0][1] + item[0][2][1]) / 2) for item in resultados]

# Sort results primarily by vertical position, then by horizontal position within small vertical tolerance
results_with_y.sort(key=lambda x: (x[1], x[0][0][0]))


# Group results into lines based on a fixed vertical tolerance
lines = []
current_line = []
line_tolerance = 15 # Adjusted to a fixed value based on visual inspection of typical text height

if results_with_y:
    current_line.append(results_with_y[0][0])
    for i in range(1, len(results_with_y)):
        item, y_center = results_with_y[i]
        prev_item, prev_y_center = results_with_y[i-1][0], results_with_y[i-1][1]

        # Check if the current item is vertically close to the previous item's line center
        if abs(y_center - prev_y_center) < line_tolerance:
            current_line.append(item)
        else:
            # Sort items within the finished line horizontally before adding
            current_line.sort(key=lambda x: x[0][0][0])
            lines.append(current_line)
            current_line = [item]

    # Add the last line
    if current_line:
        current_line.sort(key=lambda x: x[0][0][0])
        lines.append(current_line)

# Process lines to extract information
for i, line_items in enumerate(lines):
    line_text = " ".join([item[1] for item in line_items])
    line_text_upper = line_text.upper()

    # Extract F. Vencimiento and the value on the same line or the line above
    if "F.VENCIMIENTO" in line_text_upper and f_vencimiento_value is None:
        date_match = re.search(r'(\d{2}[-/]\d{2}[-/]\d{2,4})', line_text)
        if date_match:
            f_vencimiento_value = date_match.group(1)
        elif i > 0:
             prev_line_text = " ".join([item[1] for item in lines[i-1]])
             date_match = re.search(r'(\d{2}[-/]\d{2}[-/]\d{2,4})', prev_line_text)
             if date_match:
                 f_vencimiento_value = date_match.group(1)


    # Extract F. Nacimiento and Estado Civil
    if ("F.NACIMIENTO" in line_text_upper or "FNACIMIENTO" in line_text_upper) and (f_nacimiento_value is None or edo_civil_value is None):
        # Look for date pattern in the same line or the line above
        date_match = re.search(r'(\d{2}[-/]\d{2}[-/]\d{2,4})', line_text)
        if date_match and f_nacimiento_value is None:
            f_nacimiento_value = date_match.group(1)

        # Look for state civil terms on the same line or the line above
        estado_civil_match = re.search(r'(soltero|soltera|casado|casada|viudo|viuda)', line_text, re.IGNORECASE)
        if estado_civil_match and edo_civil_value is None:
            edo_civil_value = estado_civil_match.group(1)

        # If date or state civil not found on this line, check the line above
        if (f_nacimiento_value is None or edo_civil_value is None) and i > 0:
             prev_line_text = " ".join([item[1] for item in lines[i-1]])
             if f_nacimiento_value is None:
                 date_match = re.search(r'(\d{2}[-/]\d{2}[-/]\d{2,4})', prev_line_text)
                 if date_match:
                     f_nacimiento_value = date_match.group(1)
             if edo_civil_value is None:
                 estado_civil_match = re.search(r'(soltero|soltera|casado|casada|viudo|viuda)', prev_line_text, re.IGNORECASE)
                 if estado_civil_match:
                     edo_civil_value = estado_civil_match.group(1)


    # Extract Apellidos and Nombres using relative horizontal position after keywords
    for j, item in enumerate(line_items):
        texto_item = item[1]
        bbox_item = item[0]

        # Extract Apellidos
        if ("APELLIDOS" in texto_item.upper() or "APELUDOS" in texto_item.upper()) and apellidos_text is None:
            # Look for text to the right on the same line
            if j < len(line_items) - 1:
                 apellidos_text = line_items[j+1][1].strip()
                 # Simple cleaning: remove non-alphabetic characters at the start/end
                 apellidos_text = re.sub(r'^[^a-zA-Z\s]+', '', apellidos_text).strip()
                 apellidos_text = re.sub(r'[^a-zA-Z\s]+$', '', apellidos_text).strip()


        # Extract Nombres
        if ("NOMBRES" in texto_item.upper() or "NOHBRES" in texto_item.upper()) and nombres_text is None:
             # Look for text to the right on the same line
            if j < len(line_items) - 1:
                 nombres_text = line_items[j+1][1].strip()
                 # Simple cleaning: remove non-alphabetic characters at the start/end
                 nombres_text = re.sub(r'^[^a-zA-Z\s]+', '', nombres_text).strip()
                 nombres_text = re.sub(r'[^a-zA-Z\s]+$', '', nombres_text).strip()


    # Extract Numero de Identificacion (assuming it's the text after 'V' or on the next line)
    if "V" in line_text_upper and numero_identificacion_text is None:
        id_match = re.search(r"V\s*([\d.]+)", line_text)
        if id_match:
            numero_identificacion_text = id_match.group(1).replace(',', '').replace('.', '') # Remove commas and periods
        else:
             # If 'V' is found but no number immediately follows on the same line,
             # check the next line to see if it's the identification number
            if i < len(lines) - 1:
                next_line_text = " ".join([item[1] for item in lines[i+1]])
                id_match_next = re.search(r"^[\d.,]+$", next_line_text.strip()) # Check if the next line is just a number
                if id_match_next:
                    numero_identificacion_text = id_match_next.group(0).replace(',', '').replace('.', '') # Remove commas and periods


# Final check and cleaning for extracted values
if f_vencimiento_value:
    f_vencimiento_value = f_vencimiento_value.strip()
if f_nacimiento_value:
    f_nacimiento_value = f_nacimiento_value.strip()
if edo_civil_value:
    edo_civil_value = edo_civil_value.strip().capitalize() # Capitalize state civil
if apellidos_text:
    apellidos_text = apellidos_text.strip()
if nombres_text:
    nombres_text = nombres_text.strip()
if numero_identificacion_text:
    numero_identificacion_text = numero_identificacion_text.strip()


extracted_data.append({
    'F. Vencimiento': f_vencimiento_value,
    'F. Nacimiento': f_nacimiento_value,
    'Estado Civil': edo_civil_value,
    'Apellidos': apellidos_text,
    'Nombres': nombres_text,
    'Número de Identificación': numero_identificacion_text
})

# Create a pandas DataFrame
df = pd.DataFrame(extracted_data)

# Save the DataFrame to a CSV file
output_csv_file = "datos_extraidos.csv"
df.to_csv(output_csv_file, index=False, encoding='utf-8-sig')

print(f"Datos extraídos guardados en {output_csv_file}")
display(df)

Datos extraídos guardados en datos_extraidos.csv


,F. Vencimiento,F. Nacimiento,Estado Civil,Apellidos,Nombres,Número de Identificación
0,30-09-15,24-06-81,Soltero,VALERA LEAL,JAVIER ALEXANDER,14964511


**Reasoning**:
The extraction logic has been refined and executed successfully, producing a CSV file. The next step is to save the extracted data to an Excel file as required by the original task.



In [59]:
# Load the CSV file into a pandas DataFrame
csv_file_path = 'datos_extraidos.csv'
df_csv = pd.read_csv(csv_file_path)

# Define the output Excel file path
excel_file_path = 'datos_extraidos.xlsx'

# Save the DataFrame to an Excel file
df_csv.to_excel(excel_file_path, index=False)

print(f"Data successfully saved to {excel_file_path}")

Data successfully saved to datos_extraidos.xlsx
